## Busqueda hibrida

In [4]:
import os
from pathlib import Path
from typing import List
import pandas as pd
from tqdm.auto import tqdm
from openai import OpenAI
import minsearch as ms

In [1]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)


def evaluate(ground_truth, search_function, field):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q, field)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [9]:
DATA_PATH = os.getenv("DATA_PATH", "../DATASETS/faq_sacmex.csv")

def ingest_data(file_path: Path = Path(DATA_PATH), text_fields: List[str] = None):
    # Lee el archivo csv
    df = pd.read_csv(file_path)

    # Convierte los campos a string
    if text_fields:
        for field in text_fields:
            if field in df.columns:
                df[field] = df[field].astype(str)

    # Convierte el DataFrame a diccionario
    return df.to_dict(orient='records')


In [7]:
df_question = pd.read_csv('../DATASETS/ground-truth-retrieval_llama3_2_3b.csv')
ground_truth = df_question.to_dict(orient='records')
ground_truth[0]

{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
 'pregunta': '¿Dónde puedo reportar una fuga de agua?'}

In [10]:
documents = ingest_data()
documents[0]

{'id': 'd7c4ce5eda85cd602edc71a3f29193e0',
 'pregunta': '¿Dónde puedo reportar una fuga de agua?',
 'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.',
 'document': 'faq_sacmex'}

In [11]:
# El transformer
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2")

In [2]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties":
            {
                'id': {"type": "keyword"},
                'pregunta': {"type": "text"},
                'respuesta': {"type": "text"},
                'document': {"type": "text"},
                'pregunta_vector': {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
                'respuesta_vector': {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
                'preg_resp_vector': {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
            }
    }
}

index_name = "faq_sacmex"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'faq_sacmex'})

In [12]:
operations = []
embeddings = []
for doc in tqdm(documents):
    doc["pregunta_vector"] = model.encode(doc["pregunta"]).tolist()
    doc["respuesta_vector"] = model.encode(doc["respuesta"]).tolist()
    preg_resp = f"{doc["pregunta"]} {doc["respuesta"]}"
    doc["preg_resp_vector"] = model.encode(preg_resp).tolist()
    operations.append(doc)
    embeddings.append((doc["pregunta_vector"], doc["respuesta_vector"], doc["preg_resp_vector"]))

  0%|          | 0/11 [00:00<?, ?it/s]

In [13]:
# Carga de documentos
for doc in tqdm(operations):
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

  0%|          | 0/11 [00:00<?, ?it/s]

In [28]:
def elastic_search_hybrid(query):
    vector = model.encode(query)
    knn_query = {
        "field": "preg_resp_vector",
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "boost": 0.5
    }

    keyword_query = {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["pregunta", "respuesta"],
                    "type": "best_fields",
                    "boost": 0.5,
                }
            }
        }
    }

    search_query = {
        "knn": knn_query,
        "query": keyword_query,
        "size": 10,
        "_source": ["id", "pregunta", "respuesta"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [31]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        results = search_function(q['pregunta'])
        print(results[0])
        relevance = [d['id'] == q['id'] for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total)
    }

In [32]:
evaluate(ground_truth, elastic_search_hybrid)

  0%|          | 0/11 [00:00<?, ?it/s]

{'id': 'd7c4ce5eda85cd602edc71a3f29193e0', 'pregunta': '¿Dónde puedo reportar una fuga de agua?', 'respuesta': 'Debes reportarla al organismo operador de agua potable y alcantarillado de tu localidad.'}
{'id': 'c6b1388a0227cae6d6045a1dc7dd82c5', 'pregunta': '¿Qué debo hacer si veo una fuga de agua en la calle?', 'respuesta': 'En la Ciudad de México, debes reportarla al 55 5654 3210 para que una brigada del Sistema de Aguas de la Ciudad de México acuda a repararla.'}
{'id': 'b9e493259e0b9fb9f94e4f4d66ab8ded', 'pregunta': '¿Qué debo hacer si veo una fuga de agua en mi casa?', 'respuesta': 'Lo primero que debes hacer es cortar el suministro de agua, ya sea con el grifo de cierre o la válvula de cierre, que generalmente se encuentran en el baño o la cocina.\xa0'}
{'id': '5229ec6562607f2de830a7826a099964', 'pregunta': '¿Qué debo hacer si mi consumo de agua parece haberse elevado?', 'respuesta': 'Si no tienes fuga de agua en tu casa, puedes tomar la lectura del medidor, enviar una foto y com

{'hit_rate': 1.0, 'mrr': 0.9545454545454546}